In [2]:
import pandas as pd
import torch
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
from torch.utils.data import Dataset
from torchmetrics import Accuracy, F1Score




c:\Users\dell\anaconda3\envs\yarb\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [1]:
import os

os.environ['KMP_DUPLICATE_LIB_OK']='True'


In [3]:
train_path = r"C:\Users\dell\Desktop\graduation project\AI04-Grad\datasets\CREMA-D\text\train.txt"
test_path = r"C:\Users\dell\Desktop\graduation project\AI04-Grad\datasets\CREMA-D\text\test.txt"
val_path = r"C:\Users\dell\Desktop\graduation project\AI04-Grad\datasets\CREMA-D\text\val.txt"

# Map emotions to labels
emotions = {'anger': 0, 'joy': 1, 'fear': 2, 'love': 3, 'sadness': 4, 'surprise': 5}


In [4]:

# Load data from text files
def load_data(file_path):
    texts, labels = [], []
    with open(file_path, 'r', encoding='utf-8') as file:
        for line in file:
            text, label = line.strip().split(';')
            texts.append(text)
            labels.append(emotions[label])
    return texts, labels



In [5]:
# Dataset class
class EmotionDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_length=128):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]
        label = self.labels[idx]
        encoding = self.tokenizer(
            text, padding='max_length', truncation=True, max_length=self.max_length, return_tensors='pt'
        )
        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'labels': torch.tensor(label, dtype=torch.long)
        }



In [6]:

# Load datasets
train_texts, train_labels = load_data(train_path)
test_texts, test_labels = load_data(test_path)
val_texts, val_labels = load_data(val_path)


In [7]:
# Initialize tokenizer and model
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=len(emotions))



c:\Users\dell\anaconda3\envs\yarb\lib\site-packages\huggingface_hub\file_download.py:797: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification fr

In [8]:
# Create dataset objects
train_dataset = EmotionDataset(train_texts, train_labels, tokenizer)
val_dataset = EmotionDataset(val_texts, val_labels, tokenizer)
test_dataset = EmotionDataset(test_texts, test_labels, tokenizer)


In [9]:
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    acc = accuracy_score(labels, preds)
    f1 = f1_score(labels, preds, average='weighted')
    return {"accuracy": acc, "f1": f1}


In [11]:
! pip install -U accelerate


In [12]:
! pip install -U transformers


In [10]:
import accelerate

accelerate.__version__


'1.0.1'

In [10]:
import accelerate
import transformers

transformers.__version__, accelerate.__version__


('4.26.0', '1.0.1')

In [20]:
!pip install accelerate -U

In [11]:
training_args = TrainingArguments(
    output_dir='./results',
    evaluation_strategy="epoch",
    save_strategy="epoch",
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_dir='./logs',
)


In [12]:
# Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics
)


In [1]:
# Train
trainer.train()



NameError: name 'trainer' is not defined

In [13]:
import torch

# Check if CUDA (GPU support) is available
print(torch.cuda.is_available())  # This should return True if a GPU is available


True


In [14]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12,

In [18]:
from sklearn.metrics import accuracy_score, f1_score



In [19]:
val_results = trainer.evaluate(eval_dataset=val_dataset)
print("Validation Results:", val_results)

test_results = trainer.evaluate(eval_dataset=test_dataset)
print("Test Results:", test_results)


***** Running Evaluation *****
  Num examples = 2000
  Batch size = 8
546it [03:45,  2.42it/s]
***** Running Evaluation *****
  Num examples = 2000
  Batch size = 8


Validation Results: {'eval_loss': 1.7476452589035034, 'eval_accuracy': 0.353, 'eval_f1': 0.18677837002972875, 'eval_runtime': 82.0532, 'eval_samples_per_second': 24.374, 'eval_steps_per_second': 3.047}


100%|██████████| 250/250 [01:21<00:00,  3.06it/s]

Test Results: {'eval_loss': 1.7577167749404907, 'eval_accuracy': 0.3475, 'eval_f1': 0.18130971282955993, 'eval_runtime': 81.963, 'eval_samples_per_second': 24.401, 'eval_steps_per_second': 3.05}


In [22]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12,

In [23]:
# Reinitialize Trainer with new training_args
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics
)

trainer.train()


***** Running training *****
  Num examples = 16000
  Num Epochs = 5
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 10000
  Number of trainable parameters = 109486854
                                                    
  0%|          | 43/10000 [09:37<2:28:21,  1.12it/s] 

{'loss': 0.8494, 'learning_rate': 1.9e-05, 'epoch': 0.25}


                                                    
  0%|          | 43/10000 [18:11<2:28:21,  1.12it/s]  

{'loss': 0.3294, 'learning_rate': 1.8e-05, 'epoch': 0.5}


                                                    
  0%|          | 43/10000 [26:34<2:28:21,  1.12it/s]  

...



























































































































































































































































                                                       
                                                     

  0%|          | 43/10000 [10:35:54<2:28:21,  1.12it/s]

Saving model checkpoint to ./results\checkpoint-10000
Configuration saved in ./results\checkpoint-10000\config.json


{'eval_loss': 0.3201463520526886, 'eval_accuracy': 0.937, 'eval_f1': 0.9371500059594094, 'eval_runtime': 89.4094, 'eval_samples_per_second': 22.369, 'eval_steps_per_second': 2.796, 'epoch': 5.0}


Model weights saved in ./results\checkpoint-10000\pytorch_model.bin


Training completed. Do not forget to share your model on huggingface.co/models =)


                                                       
100%|██████████| 10000/10000 [10:34:52<00:00,  3.81s/it]

{'train_runtime': 38092.7651, 'train_samples_per_second': 2.1, 'train_steps_per_second': 0.263, 'train_loss': 0.16692307720184327, 'epoch': 5.0}


TrainOutput(global_step=10000, training_loss=0.16692307720184327, metrics={'train_runtime': 38092.7651, 'train_samples_per_second': 2.1, 'train_steps_per_second': 0.263, 'train_loss': 0.16692307720184327, 'epoch': 5.0})

In [27]:
# Evaluate the model on the validation dataset
eval_results = trainer.evaluate()
print("Evaluation results:", eval_results)


***** Running Evaluation *****
  Num examples = 2000
  Batch size = 8
308it [10:53:07, 127.23s/it]

Evaluation results: {'eval_loss': 0.3201463520526886, 'eval_accuracy': 0.937, 'eval_f1': 0.9371500059594094, 'eval_runtime': 85.3014, 'eval_samples_per_second': 23.446, 'eval_steps_per_second': 2.931, 'epoch': 5.0}


In [37]:
# Define your test sentence
test_sentence = "I think I am so stressed now "

# Tokenize the input sentence
inputs = tokenizer(test_sentence, return_tensors="pt", truncation=True, padding=True, max_length=128)

# Move inputs to the same device as the model
inputs = {key: value.to(device) for key, value in inputs.items()}


In [38]:
# Set the model in evaluation mode
model.eval()

# Perform prediction
with torch.no_grad():
    outputs = model(**inputs)

# Extract logits (raw prediction scores)
logits = outputs.logits
predicted_class = torch.argmax(logits, dim=-1).item()  # Get the predicted class index


In [39]:
# Reverse the emotions dictionary to map index to label
label_map = {v: k for k, v in emotions.items()}
predicted_emotion = label_map[predicted_class]

print(f"Predicted Emotion: {predicted_emotion}")


Predicted Emotion: sadness
